# ♟️ Global Chess Challenge 2025: Qwen2.5-7B Training (Exact Baseline Replica)

This notebook mirrors `train_nvidia.py` but is optimized for Kaggle/H100 and Qwen 7B.

### 🚀 Setup
1. **GPU**: Use 2x T4 or A100.
2. **Internet**: Enabled.
3. **Secrets**: Add `WANDB_API_KEY`.

In [ ]:
# 1. Install Dependencies & Repository
!pip install datasets transformers accelerate chess wandb jinja2 vllm trl peft
!pip install git+https://github.com/AIcrowd/chess-env.git
!apt-get update && apt-get install -y stockfish
# Install the baselines repo so we can import modules exactly like the script
!pip install git+https://github.com/ritwikareddykancharla/Global-Chess-Challenge-2025-Baselines.git

In [ ]:
# 2. Argument & Configuration Setup (Mirrors train_nvidia.py)
import os
# os.environ["WANDB_MODE"] = "disabled"

import torch
import pandas as pd
import argparse
from datasets import Dataset, load_dataset
import wandb
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)
from chess_evaluation_callback import ChessLLMEvaluationCallback

# --- WandB Login for Kaggle ---
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    wandb_key = user_secrets.get_secret("WANDB_API_KEY")
    wandb.login(key=wandb_key)
    print("✅ Logged into WandB using Kaggle Secret")
except Exception as e:
    print(f"⚠️ WandB auto-login failed: {e}. Make sure 'WANDB_API_KEY' is in Add-ons -> Secrets.")
    # wandb.login() # Uncomment to enter manually

# Simulate Argument Parsing for Notebook
class Args:
    output_dir = "./trained_models/qwen7b_chess_finetuned"
args = Args()

# Validate output directory argument
if os.path.exists(args.output_dir):
    if len(os.listdir(args.output_dir)) > 0:
        print(f"Warning: Output directory '{args.output_dir}' already exists and is not empty")

# --- Configuration (H100/7B Updates) ---
MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"
OUTPUT_DIR = args.output_dir
NUM_LINES_TO_LOAD = 1_000_000
MAX_LENGTH = 512  
EVAL_STEPS = 3000

BATCH_SIZE = 8 # H100 Optimized
GRAD_ACCUM_STEPS = 2
LEARNING_RATE = 5e-5 # 7B Optimized
WEIGHT_DECAY = 0.001
WARMUP_STEPS = 500
LOGGING_STEPS = EVAL_STEPS
SAVE_STEPS = EVAL_STEPS
SAVE_TOTAL_LIMIT = 10
NUM_TRAIN_EPOCHS = 1 # 1 Epoch is sufficient for 1M+ games

# Extract directory name for wandb run name
run_name = os.path.basename(os.path.normpath(OUTPUT_DIR))
wandb.init(project="ChessLLM", name=run_name)

print(f"Using device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

In [ ]:
# 3. Load Dataset
# Using the cleaner 'dataset/file' syntax as you requested
print("Loading dataset...")
dataset = load_dataset(
    "aicrowd/ChessExplained", 
    data_files="ChessExplained_2500k_qwen3.parquet",
    split="train"
).select(range(NUM_LINES_TO_LOAD))
print(f"Loaded {len(dataset)} examples")

In [ ]:
# 4. Load/Prepare Tokenizer & Model
import chess

# Create 7B Tokenizer dynamically to ensure vocabulary match
print(f"Loading tokenizer for: {MODEL_NAME}")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

# Add Special Tokens (Same logic as prepare_tokenizer.ipynb)
print("Adding chess special tokens...")
squares = [f"<{chess.square_name(sq)}>" for sq in chess.SQUARES]
pieces = [
    '<White_Pawn>', '<White_Knight>', '<White_Bishop>', 
    '<White_Rook>', '<White_Queen>', '<White_King>',
    '<Black_Pawn>', '<Black_Knight>', '<Black_Bishop>',
    '<Black_Rook>', '<Black_Queen>', '<Black_King>',
    '<blank>'
]
tokenizer.add_special_tokens({'additional_special_tokens': squares + pieces})

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"Loading model: {MODEL_NAME}")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="flash_attention_2",
)
model.resize_token_embeddings(len(tokenizer))

print(f"Model loaded with {model.num_parameters():,} parameters")

In [ ]:
# 5. Tokenize the dataset (Exact match to script)
def tokenize_function(examples):
    """Tokenize text examples"""
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        max_length=MAX_LENGTH,
        padding=False,  # We'll pad dynamically during training
        return_tensors=None
    )
    return tokenized

print("Tokenizing dataset...")
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=dataset.column_names,
    desc="Tokenizing"
)

print(f"Tokenization complete. Sample token count: {len(tokenized_dataset[0]['input_ids'])}")

In [ ]:
# 6. Training Arguments & Collator
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM_STEPS,
    gradient_checkpointing=False,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    warmup_steps=WARMUP_STEPS,
    logging_steps=LOGGING_STEPS,
    save_steps=SAVE_STEPS,
    save_total_limit=SAVE_TOTAL_LIMIT,
    bf16=True,
    fp16=False,
    optim="adamw_torch_fused",
    remove_unused_columns=False,
    report_to="wandb",
    dataloader_drop_last=True,
    num_train_epochs=NUM_TRAIN_EPOCHS,
)

print("Training configuration:")
print(f"  Batch size: {training_args.per_device_train_batch_size}")
print(f"  Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  Learning rate: {training_args.learning_rate}")

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [ ]:
# 7. Logic & Trainer Initialization
evaluation_callback = ChessLLMEvaluationCallback(
    model=model,
    tokenizer=tokenizer,
    checkpoint_dir=OUTPUT_DIR,
    eval_every_n_steps=EVAL_STEPS,
    output_dir="./eval_results_during_training",
    vllm_port=8000,
    batch_size=BATCH_SIZE,
    grad_accum_steps=GRAD_ACCUM_STEPS,
    warmup_steps=WARMUP_STEPS,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
    callbacks=[evaluation_callback]
)

print("Trainer initialized with full evaluation callback. Ready to start training.")

In [ ]:
# 8. Training Execution
try:
    print("Starting training...")

    trainer.train()

    print("\n✅ Training complete!")

    final_model_path = f"{OUTPUT_DIR}/final_model"
    print(f"Saving final model to {final_model_path}")

    trainer.save_model(final_model_path)
    tokenizer.save_pretrained(final_model_path)

    print(f"✅ Model and tokenizer saved to {final_model_path}")
except KeyboardInterrupt:
    print("Training interrupted by user")
finally:
    wandb.finish()

# 🚀 Phase 2: GRPO Training (Advanced)

This section implements **Group Relative Policy Optimization (GRPO)** using Stockfish rewards.

**Setup**: We load the SFT model we just finished training and refine it.

In [ ]:
import chess
import chess.engine
from trl import GRPOTrainer, GRPOConfig
import re

# --- 1. Persistent Stockfish Engine ---
# We keep one engine alive to save startup time
ENGINE_PATH = "/usr/games/stockfish"
# engine = chess.engine.SimpleEngine.popen_uci(ENGINE_PATH) # Initialize once

def stockfish_reward_func(prompts, completions, **kwargs):
    """Calculates reward using Stockfish Eval (Fast Depth 10)"""
    rewards = []
    # Note: In a real run, we need to extract the BOARD state from the prompt
    # For this template, we show the logic for 1 completion
    for completion in completions:
        # 1. Extract move
        match = re.search(r'<uci_move>(.*?)</uci_move>', completion)
        if not match:
            rewards.append(-1.0) # Penalty for format error
            continue
        
        move_uci = match.group(1)
        
        # 2. Logic to apply move to board would go here
        # board.push(chess.Move.from_uci(move_uci))
        
        # 3. Evaluate
        # info = engine.analyse(board, chess.engine.Limit(depth=10))
        # score = info['score'].white().score(mate_score=10000)
        
        # Placeholder reward for template
        rewards.append(0.5)
        
    return rewards

# --- 2. GRPO Configuration ---
grpo_args = GRPOConfig(
    output_dir="./qwen7b_grpo_rl",
    learning_rate=1e-6,         # Very low LR for RL
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    max_prompt_length=256,
    max_completion_length=64,
    num_generations=4,          # How many samples per prompt
    bf16=True,
)

# --- 3. Initialize Trainer ---
# model_grpo = AutoModelForCausalLM.from_pretrained(f"{OUTPUT_DIR}/final_model", ...)

# trainer = GRPOTrainer(
#     model=model_grpo,
#     reward_funcs=[stockfish_reward_func],
#     args=grpo_args,
#     train_dataset=dataset, # Requires 'prompt' column
# )

print("GRPO Template ready. Uncomment lines above to run Phase 2.")